# Example to show how previous results can be used to warmup a new tuner run

In [6]:
import random
from mango import Tuner, scheduler
from scipy.stats import dirichlet

In [1]:
import sys
import numpy as np

print("Python version:", sys.version)
print("NumPy version:", np.__version__)

Python version: 3.12.2 (main, Mar 10 2024, 17:55:21) [Clang 12.0.5 (clang-1205.0.22.9)]
NumPy version: 1.26.4


## A simple objective function that adds up all params

In [7]:
@scheduler.custom(n_jobs=3)
def objfun(batch):
    loss = []
    suc = []
    for params in batch:
        if random.choice([True, False]):
        # if True:
            l = params["a"] + params["b"] + sum(params["c"])
            loss.append(l)
            suc.append(params)
    return suc, loss


param_space = {
    "a": range(1, 100),
    "b": range(1, 100),
    "c": dirichlet([1.0] * 3),
}

tuner = Tuner(param_space, objfun)
results = tuner.run()

  0%|          | 0/20 [00:00<?, ?it/s]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Create list of tuples fro tuner results [(x, y), ...]

In [14]:
x = results["params_tried"]
y = results["objective_values"]
xy = list(zip(x, y))
xy

[({'a': 5, 'b': 34, 'c': array([0.53466935, 0.00795084, 0.45737981])}, 40.0),
 ({'a': 66, 'b': 3, 'c': array([0.78889681, 0.10480826, 0.10629493])}, 70.0),
 ({'a': 33, 'b': 12, 'c': array([0.82737897, 0.02390222, 0.14871881])}, 46.0),
 ({'a': 45, 'b': 28, 'c': array([0.44414758, 0.50245088, 0.05340154])}, 74.0),
 ({'a': 22, 'b': 71, 'c': array([0.38748376, 0.2406243 , 0.37189194])}, 94.0),
 ({'a': 79, 'b': 85, 'c': array([0.07163752, 0.83580636, 0.09255612])}, 165.0),
 ({'a': 46, 'b': 44, 'c': array([0.18365036, 0.44457032, 0.37177931])}, 91.0),
 ({'a': 91, 'b': 37, 'c': array([0.15251792, 0.02050021, 0.82698188])}, 129.0),
 ({'a': 92, 'b': 11, 'c': array([0.0804226 , 0.68308481, 0.23649258])}, 104.0),
 ({'a': 7, 'b': 22, 'c': array([0.38725705, 0.17567884, 0.43706411])}, 30.0),
 ({'a': 26, 'b': 94, 'c': array([0.85161389, 0.10951966, 0.03886646])}, 121.0),
 ({'a': 71, 'b': 99, 'c': array([0.25787283, 0.01090033, 0.73122684])}, 171.0),
 ({'a': 99, 'b': 71, 'c': array([0.03266324, 0.926

## Use the previous result

In [18]:
config = {
    "initial_random":3,
    "initial_custom": xy,
    "batch_size": 3,
    "domain_size": 1000,
    "num_iteration": 5,
}

tuner = Tuner(param_space, objfun, conf_dict=config)
results = tuner.run()

  0%|          | 0/5 [00:00<?, ?it/s]

[{'a': 89, 'b': 43, 'c': array([0.15230098, 0.56803681, 0.27966222])}, {'a': 84, 'b': 60, 'c': array([0.07626067, 0.69098341, 0.23275592])}, {'a': 96, 'b': 65, 'c': array([0.23707756, 0.20467402, 0.55824843])}]
[{'a': 46, 'b': 37, 'c': array([0.22353804, 0.72564249, 0.05081948])}, {'a': 87, 'b': 34, 'c': array([0.0043777 , 0.82006773, 0.17555456])}, {'a': 4, 'b': 42, 'c': array([0.16281066, 0.28831418, 0.54887516])}]
[{'a': 70, 'b': 47, 'c': array([0.29791352, 0.4203061 , 0.28178038])}, {'a': 3, 'b': 89, 'c': array([0.61059812, 0.12793793, 0.26146394])}, {'a': 87, 'b': 52, 'c': array([0.29149664, 0.2801235 , 0.42837986])}]
[{'a': 93, 'b': 87, 'c': array([0.5011344 , 0.46114409, 0.03772151])}, {'a': 33, 'b': 68, 'c': array([0.07382583, 0.03848381, 0.88769036])}, {'a': 43, 'b': 8, 'c': array([0.64347146, 0.34021088, 0.01631766])}]
[{'a': 98, 'b': 66, 'c': array([6.45555855e-01, 3.54006950e-01, 4.37195686e-04])}, {'a': 63, 'b': 99, 'c': array([0.19177519, 0.62151343, 0.18671138])}, {'a': 

## Confirm that the previous results were used in the tuner

In [16]:
results["params_tried"]

array([{'a': 5, 'b': 34, 'c': array([0.53466935, 0.00795084, 0.45737981])},
       {'a': 66, 'b': 3, 'c': array([0.78889681, 0.10480826, 0.10629493])},
       {'a': 33, 'b': 12, 'c': array([0.82737897, 0.02390222, 0.14871881])},
       {'a': 45, 'b': 28, 'c': array([0.44414758, 0.50245088, 0.05340154])},
       {'a': 22, 'b': 71, 'c': array([0.38748376, 0.2406243 , 0.37189194])},
       {'a': 79, 'b': 85, 'c': array([0.07163752, 0.83580636, 0.09255612])},
       {'a': 46, 'b': 44, 'c': array([0.18365036, 0.44457032, 0.37177931])},
       {'a': 91, 'b': 37, 'c': array([0.15251792, 0.02050021, 0.82698188])},
       {'a': 92, 'b': 11, 'c': array([0.0804226 , 0.68308481, 0.23649258])},
       {'a': 7, 'b': 22, 'c': array([0.38725705, 0.17567884, 0.43706411])},
       {'a': 26, 'b': 94, 'c': array([0.85161389, 0.10951966, 0.03886646])},
       {'a': 71, 'b': 99, 'c': array([0.25787283, 0.01090033, 0.73122684])},
       {'a': 99, 'b': 71, 'c': array([0.03266324, 0.92659485, 0.04074191])},
  